In [ ]:
!git clone https://github.com/BA545-SP2019/2019-competition2-3-musketeers.git
 

Cloning into '2019-competition2-3-musketeers'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 664 (delta 80), reused 89 (delta 30), pack-reused 521
Receiving objects: 100% (664/664), 20.43 MiB | 16.42 MiB/s, done.
Resolving deltas: 100% (423/423), done.


In [ ]:
%cd 2019-competition2-3-musketeers/Kevin/

/content/2019-competition2-3-musketeers/Kevin


In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 81kB 25.6MB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from Modules import *
sns.set()
%matplotlib inline
import imblearn
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

In [ ]:
#read in the data; data formated weird with extra column, 
#use only data we need
data = pd.read_csv('Data/Final_trimmed_sequential_data.csv')
df = data.iloc[:, 1:]
y = df['Y']

df.head()

AGE  Y  SEX_Female  SEX_Male  EDUCATION_Graduate School  EDUCATION_Other  \
0   24  1           1         0                          0                0   
1   26  1           1         0                          0                0   
2   34  0           1         0                          0                0   
3   37  0           1         0                          0                0   
4   57  0           0         1                          0                0   

   EDUCATION_University  MARRIAGE_Married  MARRIAGE_Non-married  \
0                     1                 1                     0   
1                     1                 0                     1   
2                     1                 0                     1   
3                     1                 1                     0   
4                     1                 1                     0   

   PERCENT_OF_LIMIT_BAL1  ...  PAY_4_Other  PAY_4_more_than_two_month_late  \
0               0.195650  ...            0                               0   
1               0.022350  ...            1                               0   
2               0.308011  ...            1                               0   
3               0.899800  ...            1                               0   
4               0.132340  ...            1                               0   

   PAY_4_on_time  PAY_4_one_month_late  PAY_5_Other  \
0              1                     0            1   
1              0                     0            1   
2              0                     0            1   
3              0                     0            1   
4              0                     0            1   

   PAY_5_more_than_two_month_late  PAY_5_on_time  PAY_6_Other  \
0                               0              0            1   
1                               0              0            0   
2                               0              0            1   
3                               0              0            1   
4                               0              0            1   

   PAY_6_more_than_two_month_late  PAY_6_on_time  
0                               0              0  
1                               1              0  
2                               0              0  
3                               0              0  
4                               0              0  

[5 rows x 53 columns]

Using scikit-optimize for BayesSearchCV of xgboost model

In [ ]:
#xgboost settings 
# SETTINGS - CHANGE THESE TO GET SOMETHING MEANINGFUL
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

Train_data, test_data = train_test_split(df, test_size = 0.2, random_state = 2019)

ITERATIONS = 200
target = 'Y'
predictors = [x for x in Train_data.columns if x not in [target]]










# Build our skopt object

Here, we'll use the BayesSearchCV class of skopt to efficiently search through the hyperparameter space to find optimal values for these hyperparameters. This acts as a drop in replacement for sklearn's GridSearchCV and RandomSearchCV. We'll compare how well this does to the `XGBosst_tuning` notebook results.

In [ ]:
# First, we specify the model we'd like to tune,
# then, we define the search spaces for each hyperparameter,
# in the structure of (minimum, maximum, format of range)
from skopt import BayesSearchCV
bayes_cv_tuner = BayesSearchCV(
    estimator = xgb.XGBClassifier(
        objective = 'binary:logistic',
        eval_metric = 'auc',
        silent=1,
        tree_method='approx'
    ),
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'min_child_weight': (0, 10),
        'max_depth': (1, 50),
        'max_delta_step': (0, 20),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'colsample_bylevel': (0.01, 1.0, 'uniform'),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'gamma': (1e-9, 0.5, 'log-uniform'),
        'min_child_weight': (0, 5),
        'n_estimators': (50, 100),
        'scale_pos_weight': (1e-6, 500, 'log-uniform')
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=2019
    ),
    n_jobs = -1,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 2019
)

In [ ]:
# helper function to see outputs 

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")

In [ ]:
#test it out

result = bayes_cv_tuner.fit(Train_data[predictors].values, Train_data[target].values, callback=status_print)

Model #1
Best ROC-AUC: 0.6426
Best params: {'colsample_bylevel': 0.6209161581689696, 'colsample_bytree': 0.9543275764819017, 'gamma': 0.019306137883859383, 'learning_rate': 0.03133533072824358, 'max_delta_step': 1, 'max_depth': 11, 'min_child_weight': 5, 'n_estimators': 57, 'reg_alpha': 0.00031596262458470826, 'reg_lambda': 0.18102189707922706, 'scale_pos_weight': 0.02920585956428371, 'subsample': 0.5387531676662357}

Model #2
Best ROC-AUC: 0.6426
Best params: {'colsample_bylevel': 0.6209161581689696, 'colsample_bytree': 0.9543275764819017, 'gamma': 0.019306137883859383, 'learning_rate': 0.03133533072824358, 'max_delta_step': 1, 'max_depth': 11, 'min_child_weight': 5, 'n_estimators': 57, 'reg_alpha': 0.00031596262458470826, 'reg_lambda': 0.18102189707922706, 'scale_pos_weight': 0.02920585956428371, 'subsample': 0.5387531676662357}

Model #3
Best ROC-AUC: 0.6426
Best params: {'colsample_bylevel': 0.6209161581689696, 'colsample_bytree': 0.9543275764819017, 'gamma': 0.019306137883859383, 

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Model #45
Best ROC-AUC: 0.7763
Best params: {'colsample_bylevel': 0.37429876941719775, 'colsample_bytree': 0.5279438775496269, 'gamma': 0.49999999999999994, 'learning_rate': 0.4357858610696449, 'max_delta_step': 5, 'max_depth': 46, 'min_child_weight': 0, 'n_estimators': 75, 'reg_alpha': 1.3761261766538657e-06, 'reg_lambda': 1000.0, 'scale_pos_weight': 59.67843001107166, 'subsample': 1.0}

Model #46
Best ROC-AUC: 0.7763
Best params: {'colsample_bylevel': 0.37429876941719775, 'colsample_bytree': 0.5279438775496269, 'gamma': 0.49999999999999994, 'learning_rate': 0.4357858610696449, 'max_delta_step': 5, 'max_depth': 46, 'min_child_weight': 0, 'n_estimators': 75, 'reg_alpha': 1.3761261766538657e-06, 'reg_lambda': 1000.0, 'scale_pos_weight': 59.67843001107166, 'subsample': 1.0}

Model #47
Best ROC-AUC: 0.7763
Best params: {'colsample_bylevel': 0.37429876941719775, 'colsample_bytree': 0.5279438775496269, 'gamma': 0.49999999999999994, 'learning_rate': 0.4357858610696449, 'max_delta_step': 5, '

In [ ]:
#since last rounds gave minimal improvement,
#we'll use last rounds params

xgb_best = xgb.XGBClassifier(colsample_bylevel= 0.43137229639036795, 
                              colsample_bytree= 0.4480643933948192, 
                              gamma= 0.011571057538450112, 
                              learning_rate= 0.3200409629730479, 
                              max_delta_step= 0, 
                              max_depth= 36, 
                              min_child_weight= 1, 
                              n_estimators= 92, 
                              reg_alpha= 1.3963892344511292e-09, 
                              reg_lambda= 1000.0, 
                              scale_pos_weight= 1.1507206056795551, 
                              subsample= 1.0)


In [ ]:
#now we will train and make predictions


clf = xgb_best

clf.fit(Train_data[predictors], Train_data[target])

train_predictions = clf.predict(Train_data[predictors])

#train accuracy, auc, f1 score
print(f"Train Accuracy Score: {metrics.accuracy_score(Train_data[target], train_predictions)}")
print(f"Train AUC Score: {metrics.roc_auc_score(Train_data[target], train_predictions)}")
print(f"Train F1 Score: {metrics.f1_score(Train_data[target], train_predictions)}")

      

Train Accuracy Score: 0.8283333333333334
Train AUC Score: 0.6780529404452781
Train F1 Score: 0.5136921624173749


In [ ]:
# see the results on the test set
test_predictions = clf.predict(test_data[predictors])

#train accuracy, auc, f1 score
print(f"Test Accuracy Score: {metrics.accuracy_score(test_data[target], test_predictions)}")
print(f"Test AUC Score: {metrics.roc_auc_score(test_data[target], test_predictions)}")
print(f"Test F1 Score: {metrics.f1_score(test_data[target], test_predictions)}")

Test Accuracy Score: 0.8275
Test AUC Score: 0.6694785957635906
Test F1 Score: 0.4943820224719101


In [ ]:
%pwd

'/content/2019-competition2-3-musketeers/Kevin'

In [ ]:
#does it beat the best xgboost model from before? No
from sklearn.externals import joblib

xgb4 = joblib.load('Models/xgboost4.dat')

y_pred = xgb4.predict(test_data[predictors])
print(f"Test Accuracy Score: {metrics.accuracy_score(test_data[target], y_pred)}")
print(f"Test AUC Score: {metrics.roc_auc_score(test_data[target], y_pred)}")
print(f"Test F1 Score: {metrics.f1_score(test_data[target], y_pred)}")

Test Accuracy Score: 0.8133333333333334
Test AUC Score: 0.7071742457907471
Test F1 Score: 0.5454545454545454
